Importando todas as bibliotecas e bases de dados de abono e aposentadoria:

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

In [21]:
from script_tratamento_abono import tratamento_abono

time_series_abono = []

for year in ['2017', '2018', '2019', '2020', '2021']:
    for month in ['01', '07']:
        filename = f"{month}_{year}"
        base = tratamento_abono(filename)
        time_series_abono.append(base)

In [22]:
from script_tratamento_aposentados import tratamento_aposentados

time_series_aposentadoria = []
filenames = ['01_2017', '07_2017', '01_2018', '07_2018','01_2019', '06_2019', '05_2020', '07_2021', '11_2021']

for file in filenames:
    df = tratamento_aposentados(file)
    time_series_aposentadoria.append(df)

## Cruzando as bases de dados

Cruzando a base de dados de abono com a de aposentadoria, obtendo uma base de pessoas que receberam abono e já se aposentaram

In [23]:
df = time_series_abono[0]
aposentadoria_df_completo = pd.concat(time_series_aposentadoria)
full_matched_df = df.merge(aposentadoria_df_completo, on="CPF")

Acrescentando a média de valor dos cargos

In [24]:
media_dos_cargos = dict(full_matched_df.groupby(full_matched_df.descricao_cargo).valor_aposentadoria.mean())

medias = []
for cargo in full_matched_df.descricao_cargo:
    medias.append(int(media_dos_cargos[cargo]))

full_matched_df['media_cargo'] = medias

In [25]:
media_dos_estados = dict(full_matched_df.groupby(full_matched_df.UF_residencia).valor_aposentadoria.mean())

medias = []
for uf in full_matched_df.UF_residencia:
    medias.append(int(media_dos_estados[uf]))

full_matched_df['media_UF'] = medias

Depois, extraímos somente as colunas que nos interessam e exportamos o csv

In [26]:
feature_names = ['nome_x', 'escolaridade', 'media_UF', 'anos_servico', 'media_cargo','valor_abono', 'valor_aposentadoria']
full_matched_df = full_matched_df.filter(items=feature_names)
full_match_df = full_matched_df.rename(columns={'nome_x': 'nome'})

In [27]:
full_matched_df.to_csv("servidores.csv")